# ELISAs using bacterially-produced recombinant VHHs identified by solid-phase panning. (`fig-suppl-brvhhs`)

In [1]:
import pandas as pd
import altair as alt
from natsort import natsort_keygen

In [2]:
FONT_SIZE=6

### Standard ELISA using brVHHs

In [3]:
%%bash
out=../output/fig-suppl-brvhhs/
rm -rf "$out"
mkdir -p "$out"

In [4]:
CG025ae_af = pd.read_csv("../data/fig-panning-proteins/2021-08-17_CG025af_rVHH_ELISA/CG025ae_CG025af.csv", index_col=0)

In [5]:
CG025ae_af = CG025ae_af.sort_values("clone", key=natsort_keygen())
CG025ae_af.bait_antigen = CG025ae_af.bait_antigen.replace({"+": "Antigen+ protein", "-": "Antigen- protein"})
CG025ae_af["clone_antigen"] = (CG025ae_af["antigen"] + "\n" + CG025ae_af["clone"]).str.split("\n")

In [6]:
sorter_list = ["PAK flagellin", "PAO1 flagellin", "PAK pilin"]

sorter = dict(zip(sorter_list, list(range(len(sorter_list)))))

CG025ae_af = CG025ae_af.sort_values(by="antigen", key=lambda column: column.map(lambda e: sorter_list.index(e)))
# CG025ae_af = CG025ae_af.sort_values(by="antigen", key=lambda column: column.map(lambda e: sorter.index(e)))
CG025ae_af = CG025ae_af.sort_values(by="antigen", key=lambda column: column.replace(sorter))
# CG025ae_af.groupby("antigen").sort_values(by="clone")

In [7]:
(alt.Chart(CG025ae_af)
 .mark_point()
 .encode(
     x=alt.X("concentration:O", title=None,
             # continuous log scale interferes with facet plot width
             # scale=alt.Scale(type="log"), 
            ),
     y=alt.Y("A450", title="A450"),
     color=alt.Color("bait_antigen", title="Well contents", sort=["Antigen- protein","Antigen+ protein"]),
     xOffset="jitter:Q",
 ).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    # jitter='random()'
 ).properties(
     height=100
 )
 .facet(
     column=alt.Column("clone_antigen",
#                        sort=[
#                            "PAK flagellin\n1.3.5",
#                            "PAK flagellin\n1.7.1",
#                            "PAK flagellin\nNo 1°",
#                            "PAK flagellin\nYU573",
                           
#                            "PAO1 flagellin\n2.5.6",
#                            "PAO1 flagellin\n2.13.1",
#                            "PAO1 flagellin\nNo 1°",
#                            "PAO1 flagellin\nYU573",
                                                      
#                            "PAK pilin\n4.1.2",
#                            "PAK pilin\n4.2.1",
#                            "PAK pilin\nNo 1°",
#                            "PAK pilin\nYU586",
#                        ],
                       title=None)
 ).resolve_scale(
     x='independent',
     xOffset="independent"
 ).configure(
     view=dict(
         discreteWidth=dict(step=FONT_SIZE)
     ),
     facet=dict(
        spacing=FONT_SIZE//2
     ),
     point=dict(
         size=FONT_SIZE,
         strokeWidth=2,
     ),
     header=dict(
         labelFontSize=FONT_SIZE,
         titleFontSize=FONT_SIZE,
         labelPadding=FONT_SIZE//2,
     ),
     axis=dict(
         labelFontSize=FONT_SIZE,
         titleFontSize=FONT_SIZE
     ),
     legend=dict(
         titleFontSize=FONT_SIZE,
         labelFontSize=FONT_SIZE,
         symbolSize=FONT_SIZE*2,
         orient="bottom"
     )
 )
)

alt.FacetChart(...)

In [8]:
df = (CG025ae_af[['antigen', 'clone','bait_antigen','concentration','A450']]
 .rename({
     'clone': 'brVHH clone or primary antibody',
     'bait_antigen': 'bait well contents',
     'concentration': 'concentration µg/mL',
 })
).copy()
df["clone"] = df["clone"].replace({
    "1.3.5" :  "brVHH 1.3.5" ,
    "1.7.1" :  "brVHH 1.7.1" ,
    "2.5.6" :  "brVHH 2.5.6" ,
    "2.13.1" : "brVHH 2.13.1",
    "4.1.2" :  "brVHH 4.1.2" ,
    "4.2.1" :  "brVHH 4.2.1" ,
})
df.to_csv('../output/fig-suppl-brvhhs/brvhhs-standard-elisas-dilution.csv', index=False)

In [9]:
print(f"n={CG025ae_af.groupby(['clone','antigen','bait_antigen','concentration'])['A450'].count().min()} replicates per sample")

n=3 replicates per sample


### Fixed cell-based ELISA

In [10]:
CG025ai = pd.read_csv("../data/fig-panning-proteins/2021-10-01_CG025ai_fixed_cell_ELISA/CG025ai.csv", index_col=0)

In [11]:
CG025ai = CG025ai.sort_values("clone", key=natsort_keygen())
CG025ai.bait_antigen = CG025ai.bait_antigen.replace({"+": "Antigen+ cells", "-": "Antigen- cells"})
CG025ai["clone"] = CG025ai["clone"].replace({
    "rVHH 1.3.5" : "brVHH 1.3",
    "rVHH 1.7.1" : "brVHH 1.7",
    "rVHH 2.5.6" : "brVHH 2.5",
    "rVHH 2.13.1" : "brVHH 2.13",
    "rVHH 4.1.2" : "brVHH 4.1",
    "rVHH 4.2.1" : "brVHH 4.2",
})
CG025ai = CG025ai.query(
    "(clone != 'No 1°+No 2°') & "
    "((dilution != '1/100' & clone.str.startswith('YU')) |"
    " (dilution != '1/1000' & ~clone.str.startswith('YU')))")

In [12]:
(alt.Chart(CG025ai)
 .mark_point()
 .encode(
     x=alt.X("clone", title=None, sort=None),
     y=alt.Y("A450", title="A450"),
     color=alt.Color("bait_antigen", title="Well contents", sort=["Antigen- cells","Antigen+ cells"]),
     xOffset="jitter:Q",
 ).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    # jitter="0.1*sqrt(-2*log(random()))*cos(2*PI*random())"
    # jitter='random()/100'
     jitter='0'
 ).properties(
     width=50, height=100
 )
 .facet(
     column=alt.Column("antigen", sort=["PAK flagella", "PAO1 flagella", "PAK pili"], title=None)
 ).resolve_scale(
     x='independent',
     xOffset="independent",
     # xOffset="shared"
 ).configure(
     view=dict(
         discreteWidth=dict(step=0.5*FONT_SIZE)
     ),
     facet=dict(
        spacing=FONT_SIZE//2
     ),
     point=dict(
         size=FONT_SIZE,
         strokeWidth=2,
     ),
     header=dict(
         labelFontSize=FONT_SIZE,
         titleFontSize=FONT_SIZE,
         labelPadding=FONT_SIZE//2,
     ),
     axis=dict(
         labelFontSize=FONT_SIZE,
         titleFontSize=FONT_SIZE
     ),
     legend=dict(
         titleFontSize=FONT_SIZE,
         labelFontSize=FONT_SIZE,
         symbolSize=FONT_SIZE*2,
         orient="bottom"
     )
 )
)

alt.FacetChart(...)

In [13]:
(CG025ai[['antigen','clone','dilution','bait_antigen','A450']]
 .rename({
     'clone': 'brVHH clone or primary antibody',
     'bait_antigen': 'bait well contents'
 })
 .to_csv('../output/fig-suppl-brvhhs/brvhhs-fixed-cell-based-elisas.csv', index=False)
)